In [1]:
import pandas as pd
import requests
from tqdm import tqdm
import numpy as np
import json
import itertools
from bs4 import BeautifulSoup

C:\Users\Afag\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [66]:
r = requests.get('https://bravomarket.online/catalog/dairy-and-eggs/cream/petmol-duru-qaymaq-33-500-qr/')
soup = BeautifulSoup(r.text, 'html.parser')

In [24]:
def scrape_item(url): 
    a = requests.get(url).text
    soup= BeautifulSoup(a, 'html.parser')
    try:
        title = soup.find('h1', attrs={'class':'mb-1'}).text
    except:
        title = np.nan
    try:
        price = soup.find('div', attrs = {'class': 'product-detail-price-item product-item-detail-price-current'}).text.strip().split('\n')[0]
    except:
        price = np.nan
    try: 
        if title.endswith('KQ') or title.endswith('QR') or title.endswith('GR'):
            quantity = title.split()[-1]
        else:
            quantity = np.nan
    except:
        quantity = np.nan
    cat =  soup.find_all('div', attrs = {'class': 'breadcrumb-item breadcrumbs_item'})[-2:-1]
    try:
        for i in cat:
            categ = i.find('a')['title']
    except:
        categ= np.nan
    subcat =  soup.find_all('div', attrs = {'class': 'breadcrumb-item breadcrumbs_item'})[-2:]
    try:
        for i in subcat:
            subcateg = i.find('a')['title']
    except:
        subcateg =  np.nan
#     try:
#         ceki = soup.find_all('option')[1].text[:4].strip()
#         olcu = soup.find_all('option')[1].text[3:5].strip()
#     except: 
#         ceki = 1
#         olcu = 'kq'
    dict1 = {'product': title, 
            'price': price, 
            'quantity': quantity,
             'category' : categ, 
             'subcategory': subcateg
            }
    return dict1

In [3]:
scrape_item('https://bravomarket.online/catalog/savoury-grocery/rice_pasta_pulses1/m-u-t-l-u-g-r-a-n-d-e-f-u-s-i-l-l-i-5-0-0-g-r-x-2-0/')

{'product': 'MUTLU MAKARON SPİRAL GRANDE 500QR',
 'price': '₼ 1.49',
 'quantity': '500QR',
 'category': 'BAQQALİYƏ',
 'subcategory': 'DÜYÜ MAKARON VƏ PAXLALAR'}

## Category links

In [4]:
r = requests.get('https://bravomarket.online/catalog')
soup = BeautifulSoup(r.text, 'html.parser')
category_links = []
for i in soup.find_all('div',
                       attrs = {'class': 'catalog-section-list-tile-img-container embed-responsive embed-responsive-16by9'}):
    category_links.append('https://bravomarket.online' + i.find('a')['href'])

## Subcategory links

In [7]:
subcategory_links = []
r = requests.get('https://bravomarket.online/catalog/dairy-and-eggs/')
soup = BeautifulSoup(r.text, 'html.parser')
for i in soup.find_all('div',attrs = {'class': 'section__list-wrapper'}):
    for i in i.find_all('a'):
        subcategory_links.append('https://bravomarket.online' + i['href'])

In [162]:
for i in subcategory_links:
    r = requests.get(i)
    soup = BeautifulSoup(r.text, 'html.parser')

https://bravomarket.online/catalog/dairy-and-eggs/butter/
https://bravomarket.online/catalog/dairy-and-eggs/cream/
https://bravomarket.online/catalog/dairy-and-eggs/curd/
https://bravomarket.online/catalog/dairy-and-eggs/drinking-dairy-products/
https://bravomarket.online/catalog/dairy-and-eggs/e-g-g-s/
https://bravomarket.online/catalog/dairy-and-eggs/flavored-yoghurts/
https://bravomarket.online/catalog/dairy-and-eggs/mayonnaise/
https://bravomarket.online/catalog/dairy-and-eggs/milk/
https://bravomarket.online/catalog/dairy-and-eggs/plain-yoghurts/
https://bravomarket.online/catalog/dairy-and-eggs/sour-cream/
https://bravomarket.online/catalog/dairy-and-eggs/sourcream/
https://bravomarket.online/catalog/dairy-and-eggs/white-cheese/
https://bravomarket.online/catalog/dairy-and-eggs/yellow-cheese/


In [143]:
r = requests.get('https://bravomarket.online/catalog/heavy-household/small-appliance/')
soup = BeautifulSoup(r.text, 'html.parser')

## Scraping all links in one page

In [9]:
def scraping_one_page(subcategory_url): 
    r = requests.get(subcategory_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    xeber_linkleri = []
    for i in soup.find_all('div', attrs = {'class': 'product_item_title'}):
        xeber_linkleri.append('https://bravomarket.online' + i.find('a')['href'])
    return xeber_linkleri

In [11]:
def scrape_all_links_in_category(category_url): 
    r = requests.get(category_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    subcategory_links = []
    for i in soup.find_all('div',attrs = {'class': 'section__list-wrapper'}):
        for i in i.find_all('a'):
            subcategory_links.append('https://bravomarket.online' + i['href'])
    pages_of_subcategory = []
    pages_of_news = []
    for link in tqdm(subcategory_links):
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'html.parser')
        pages = []
        try:
            for i in soup.find_all('ul', attrs = {'class': 'bx_pagination_page_list_num'}):
                for i in i.find_all('li'):
                    pages.append(i.text)
            last_page = int(pages[-1])

            for number in range(1,last_page + 1):
                pages_of_subcategory.append(f'{link}?PAGEN_1={number}&SIZEN_1=12')
        except:
            pages_of_subcategory.append(link)
        for url in pages_of_subcategory:
            pages_of_news.append(scraping_one_page(url))
    pages_of_news = list(itertools.chain.from_iterable(pages_of_news))
    return list(set(pages_of_news))


In [12]:
all_links_in_website = []
for i in tqdm(category_links): 
    all_links_in_website.append(scrape_all_links_in_category(i))

100%|███████████████████████████████████████████████████████████████████████████████| 33/33 [1:05:47<00:00, 119.61s/it]


In [13]:
all_links_in_website = list(itertools.chain.from_iterable(all_links_in_website))

In [23]:
all_product = []
for i in tqdm(all_links_in_website): 
    all_product.append(scrape_item(i))

 51%|█████████████████████████████████████▋                                    | 1500/2942 [1:35:37<1:31:56,  3.83s/it]


UnboundLocalError: local variable 'categ' referenced before assignment

In [26]:
for i in tqdm(all_links_in_website[1500:]): 
    all_product.append(scrape_item(i))

100%|████████████████████████████████████████████████████████████████████████████| 1442/1442 [1:21:28<00:00,  3.39s/it]


In [28]:
len(all_product)

2942

In [29]:
final_df = pd.DataFrame(all_product)

In [34]:
final_df[final_df['product'].isnull()]

,product,price,quantity,category,subcategory
1282,NaN,NaN,NaN,Kataloq,EV HEYVANLARI ÜÇÜN QİDA


In [31]:
final_df.isnull().sum()

product           1
price             1
quantity       1941
category          0
subcategory       0
dtype: int64